In [21]:
import tensorflow as tf
import %pip install tensorflow_decision_forests
tensorflow_decision_forests as tfdf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
print("TensorFlow v" + tf.__version__)
print("TensorFlow Decision Forests v" + tfdf.__version__)

In [11]:
df = pd.read_csv("train.csv")
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [25]:
df = pd.read_csv('train.csv')
def preprocessing(df):

    def calculate_group_size(passenger_id, df):
        # Extract the prefix from the 'PassengerId'
        prefix = passenger_id.split('_')[0]
        
        # Count the occurrences of the same prefix in the entire DataFrame
        group_size = len(df[df['PassengerId'].str.startswith(prefix)])
        
        return group_size

    # Apply the function to create the 'TravelGroupSize' column
    df['TravelGroupSize'] = df.apply(lambda row: calculate_group_size(row['PassengerId'], df), axis=1)

    # Filling missing values for HomePlanet
    df['HomePlanet'] = df['HomePlanet'].fillna('Missing')
    df['HomePlanet'] = df['HomePlanet'].replace('Missing', -1)

    # Filling missing values for CryoSleep
    df['CryoSleep'] = df['CryoSleep'].fillna('Missing')
    df['CryoSleep'] = df['CryoSleep'].replace('Missing', -1)

    # Filling missing values for Cabin
    df['Cabin'] = df['Cabin'].fillna('Missing')
    
    # Cabin split
    mask = df['Cabin'] != "-1"

    # Split 'Cabin' into 'Deck', 'Num', and 'Side' columns
    split_values = df['Cabin'].str.split('/', expand=True)

    # Assign the split values to 'Deck' and 'Side' for non-"Unknown" rows
    df.loc[mask, 'Deck'] = split_values[0]
    df.loc[mask, 'Num'] = split_values[1]
    df.loc[mask, 'Side'] = split_values[2]

    # Fill missing
    df['Deck'] = df['Deck'].fillna("Missing")
    df['Num'] = df['Num'].fillna("Missing")
    df['Side'] = df['Side'].fillna("Missing")
    df['Deck'] = df['Deck'].replace('Missing', -1)
    df['Num'] = df['Num'].replace('Missing', -1)
    df['Side'] = df['Side'].replace('Missing', -1)

    # Filling missing values for Destination
    df['Destination'] = df['Destination'].fillna('Missing')
    df['Destination'] = df['Destination'].replace('Missing', -1)

    # Filling missing values for Age
    df['Age'] = df['Age'].fillna(df['Age'].median())

    # Filling missing values for VIP
    df['VIP'] = df['VIP'].fillna(False)

    # Filling missing values for amenities
    df[['FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'RoomService']] = df[['FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'RoomService']].fillna(0)
    df['CryoSleep'] = df['CryoSleep'].astype(int)
    df['VIP'] = df['VIP'].astype(int)
    df['Transported'] = df['Transported'].astype(int)




    # Less important features
    df = df.drop(columns=['Name', 'PassengerId', 'Cabin'])
    df = df.astype({
        'CryoSleep': 'int',
        'VIP': 'int16',
        'FoodCourt': 'int16',
        'ShoppingMall': 'int16',
        'Spa': 'int16',
        'VRDeck': 'int16',
        'RoomService': 'int16',
        'Age': 'int8',
        'TravelGroupSize': 'int8',
        'Transported': 'int16'
    })

    return df

# Assuming you have a DataFrame named 'df' that you want to preprocess
#change path if needed for concat file
df = preprocessing(df)
df.to_csv('train_preprocessed_tf.csv', index=False)

In [33]:
df.dtypes

HomePlanet         object
CryoSleep           int32
Destination        object
Age                  int8
VIP                 int32
RoomService         int16
FoodCourt           int16
ShoppingMall        int16
Spa                 int16
VRDeck              int16
Transported         int32
TravelGroupSize      int8
Deck               object
Num                object
Side               object
dtype: object

7043 examples in training, 1650 examples in testing.


In [34]:

# Load the preprocessed DataFrame

# Split the data into features (X) and target variable (y)
X = df.drop(columns=["Transported"])
y = df["Transported"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a binary classification model using TensorFlow/Keras
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(len(X.columns),)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).